# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [65]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url ="https://country-leaders.onrender.com"
# assign the /status endpoint to another variable called status_url (1 line)
endpoint = "status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{endpoint}")
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
        content = req.json()
        print(content)
else:
    print("🚫 Error {req.status_code} : {req.reason}")

Alive


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [66]:
# Set the countries_url variable (1 line)
countries_url = "countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req_countries = requests.get(f"{root_url}/{countries_url}")
# Get the JSON content and store it in the countries variable (1 line)
countries = req_countries.json()
# display the request's status code and the countries variable (1 line)
print(req_countries.status_code, countries)

403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [67]:
# Set the cookie_url variable (1 line)
cookie_url = "cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
response = requests.get(f"{root_url}/{cookie_url}")
cookies = response.cookies.get_dict()
print("🍪 Cookies:", cookies)


🍪 Cookies: {'user_cookie': 'bec2c01e-6a7a-4b63-a682-cc18939686ee'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [68]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
cookie = requests.get(f"{root_url}/{cookie_url}").cookies.get_dict()
req = requests.get(f"{root_url}/{countries_url}", cookies=cookie)
# display the countries variable (1 line)
countries = req.json()
print(req.status_code, countries)

200 ['fr', 'be', 'ma', 'us', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [69]:
# Set the leaders_url variable (1 line)
leaders_url = "leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
req = requests.get(f"{root_url}/{leaders_url}", cookies= requests.get(f"{root_url}/{cookie_url}").cookies.get_dict())
# display the leaders variable (1 line)
print(req.status_code, req.reason)


404 Not Found


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [70]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
cookie_response = requests.get(f"{root_url}/{cookie_url}")
cookies = cookie_response.cookies.get_dict()
req = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={"country": "fr"} )
# assign the output to the leaders variable (1 line)
leader= req.json()
# display the leaders variable (1 line)
print(leader)


[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [71]:
# 4 lines
cookie_response = requests.get(f"{root_url}/{cookie_url}")
cookies = cookie_response.cookies.get_dict()
for country in countries:
    req = requests.get(f"{root_url}/{leaders_url}", cookies=cookies, params={"country": country} )
leaders_per_country = req.json()
print(leaders_per_country)


[{'id': 'Q7747', 'first_name': 'Vladimir', 'last_name': 'Putin', 'birth_date': '1952-10-07', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87', 'start_mandate': '2000-05-07', 'end_mandate': '2008-05-07'}, {'id': 'Q23530', 'first_name': 'Dmitry', 'last_name': 'Medvedev', 'birth_date': '1965-09-14', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87', 'start_mandate': '2008-05-07', 'end_mandate': '2012-05-07'}, {'id': 'Q34453', 'first_name': 'Boris', 'last_name': 'Yeltsin', 'birth_date': '1931-02-01', 'death_date': '2007-04-23', 'place_of_birth': 'Butka', 'wikipedia_url

In [72]:
# or 1 line
leaders_per_country = req = requests.get(f"{root_url}/{leaders_url}", cookies=requests.get(f"{root_url}/{cookie_url}").cookies.get_dict(), params={"country": (country for country in countries) } ).json()
# print(leaders_per_country)

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [73]:
# < 15 lines
def get_leaders():
    # define the urls
    root_url ="https://country-leaders.onrender.com"
    cookie_url = "cookie"
    countries_url = "countries"
    leaders_url = "leaders"
    # get the cookies
    cookie_response = requests.get(f"{root_url}/{cookie_url}")
    cookies = cookie_response.cookies.get_dict()
    # get the countries
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    # loop over them and save their leaders in a dictionary
    leaders_per_country  = {}
    for country in countries:
        leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies,params={"country":country}).json()
        leaders_per_country[country] = leader
    return leaders_per_country 


    

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [74]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

{'fr': [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_u

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [75]:
# 3 lines
wikipedia_url = leaders_per_country["fr"][0]["wikipedia_url"]
req = requests.get(wikipedia_url)
print(req.text)
print(wikipedia_url)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="fr" dir="ltr">
<head>
<meta charset="UTF-8">
<title>François Hollande — Wikipédia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [76]:
# 3 line
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, "html")
soup_text= soup.get_text()
print(soup_text.strip())
# print(soup.prettify())get_text(prettify())

François Hollande — Wikipédia



































Aller au contenu







Menu principal





Menu principal
déplacer vers la barre latérale
masquer



		Navigation
	


Page d’accueilPortails thématiquesArticle au hasardContact





		Contribuer
	


Débuter sur WikipédiaAideCommunautéPages spécialesModifications récentes



















Rechercher











Rechercher






















Apparence
















Faire un don

Créer un compte

Se connecter








Outils personnels





Faire un don Créer un compte Se connecter





		Pages pour les contributeurs déconnectés en savoir plus



ContributionsDiscussion




























Sommaire
déplacer vers la barre latérale
masquer




Début





1
Situation personnelle




Afficher / masquer la sous-section Situation personnelle





1.1
Famille








1.2
Vie privée








1.3
Formation








1.4
Carrière professionnelle










2
Parcours politique




Afficher / masquer la sous-section Parcours 

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [77]:
# 2 lines
paragraphs = soup.find_all("p")
print(paragraphs)

[<p>Pour les articles homonymes, voir <a class="mw-disambig" href="/wiki/Hollande_(homonymie)" title="Hollande (homonymie)">Hollande (homonymie)</a>.
</p>, <p class="mw-empty-elt">
</p>, <p><b>François Hollande</b> <span class="API nowrap" style="font-family:'Segoe UI','DejaVu Sans','Lucida Grande','Lucida Sans Unicode','Arial Unicode MS','Hiragino Kaku Gothic Pro',sans-serif;" title="Alphabet phonétique international">[<a class="mw-redirect" href="/wiki/API_f" title="API f"><span title="[f] « f » dans « fête ».">f</span></a><a class="mw-redirect" href="/wiki/API_%CA%81" title="API ʁ"><span title="[ʁ] « r » dans « roue ».">ʁ</span></a><a class="mw-redirect" href="/wiki/API_%C9%91%CC%83" title="API ɑ̃"><span title="[ɑ̃] « an » dans « sans ».">ɑ̃</span></a><a class="mw-redirect" href="/wiki/API_s" title="API s"><span title="[s] en français « s » dans « sa » ; ailleurs avec la langue un peu plus en arrière (tel le basque : « s » dans « euskara »).">s</span></a><a class="mw-redirect" href=

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [78]:
# <10 lines
# paragraphs = soup.find_all("p")
# first_paragraph = paragraphs[2].get_text()
# print(first_paragraph)

all_paragraphs = soup.find_all("p")
for paragraph in all_paragraphs:
     if paragraph.get("class") is None and not paragraph.find_parents("div", class_="bandeau-cell"):
        first_paragraph = paragraph.get_text()
        break 
print(first_paragraph)
    


François Hollande [fʁɑ̃swa ɔlɑ̃d][n 3] Écouterⓘ, né le 12 août 1954 à Rouen (Seine-Inférieure), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [79]:
# 10 lines
import re
def replace_eng(url):
   en_url = re.sub(r'https?://[a-z]{2}\.', "https://en.", url)
   return en_url

def get_first_paragraph(wikipedia_url):
  print(wikipedia_url) # keep this for the rest of the notebook
  wikipedia_eng_url = replace_eng(wikipedia_url)
  print(wikipedia_eng_url)
  response = requests.get(wikipedia_eng_url)
  soup = BeautifulSoup(response.text, "html.parser")
  all_paragraphs = soup.find_all("p")
  for paragraph in all_paragraphs:
     if paragraph.get("class") is None and not paragraph.find_parents("div", class_="bandeau-cell"):
        first_paragraph = paragraph.get_text()
        break 
  return first_paragraph


In [80]:
# Test: 3 lines
for country in countries:
    for leader in leaders_per_country[country]:
        wikipedia_url = leader["wikipedia_url"]
        first_paragraph = (get_first_paragraph(wikipedia_url))
        print(first_paragraph)
        
    
        



https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
https://en.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Gérard Georges Nicolas Hollande (French: [fʁɑ̃swa ʒeʁaʁ ʒɔʁʒ(ə) nikɔla ɔlɑ̃d] ⓘ; born 12 August 1954) is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party (PS) from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.

https://fr.wikipedia.org/wiki/Nicolas_Sarkozy
https://en.wikipedia.org/wiki/Nicolas_Sarkozy
Nicolas Paul Stéphane Sarközy de Nagy-Bocsa (/sɑːrˈkoʊzi/ sar-KOH-zee; French: [nikɔla pɔl stefan saʁkɔzi də naʒi bɔksa] ⓘ; born 28 January 1955) is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilt

### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [81]:
# 3 lines
# txt = "The rain in Spain"
# x = re.sub("\s", "9", txt)

def clean_p(p):
    cleaned = re.sub(r'\[.*?\]|\(.*?\)', '', p)
    return cleaned





Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [82]:
# 10 lines
def get_first_paragraph(wikipedia_url):
  print(wikipedia_url) # keep this for the rest of the notebook
  wikipedia_eng_url = replace_eng(wikipedia_url)
  print(wikipedia_eng_url)
  response = requests.get(wikipedia_eng_url)
  soup = BeautifulSoup(response.text, "html.parser")
  all_paragraphs = soup.find_all("p")
  for paragraph in all_paragraphs:
     if paragraph.get("class") is None and not paragraph.find_parents("div", class_="bandeau-cell"):
        first_paragraph = paragraph.get_text()
        clean_paragraph = clean_p(first_paragraph)
        break 
  return clean_paragraph

for country in countries:
    for leader in leaders_per_country[country]:
        wikipedia_url = leader["wikipedia_url"]
        first_paragraph = (get_first_paragraph(wikipedia_url))
        print(first_paragraph)


https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
https://en.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Gérard Georges Nicolas Hollande  nikɔla ɔlɑ̃d] ⓘ; born 12 August 1954) is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party  from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.

https://fr.wikipedia.org/wiki/Nicolas_Sarkozy
https://en.wikipedia.org/wiki/Nicolas_Sarkozy
Nicolas Paul Stéphane Sarközy de Nagy-Bocsa  is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilty of having tried to bribe a judge in 2014 to obtain information and spending beyond legal campaign funding limits during his 2012 re-el

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [104]:
# < 20 lines
from requests import Session
def clean_text(p):
    cleaned = re.sub(r'\[.*?\]', "", p)
    cleaned = re.sub(r'\[[^\]]*\d+[^\]]*\]', "", cleaned)
    cleaned = re.sub(r'\(.*?\)', "", cleaned)
    cleaned = re.sub(r'[ⓘ;)]+|\s+', " ", cleaned).strip() 
    return cleaned

def get_first_paragraph(wikipedia_url):
  # print(wikipedia_url) # keep this for the rest of the notebook
  wikipedia_eng_url = replace_eng(wikipedia_url)
  # print(wikipedia_eng_url)
  session = requests.Session()
  response = session.get(wikipedia_eng_url)
  soup = BeautifulSoup(response.text, "html.parser")
  all_paragraphs = soup.find_all("p")
  for paragraph in all_paragraphs:
     if paragraph.get("class") is None and not paragraph.find_parents("div", class_="bandeau-cell"):
        first_paragraph = paragraph.get_text()
        clean_paragraph = clean_text(first_paragraph)
        break 
  return clean_paragraph

for country in countries:
    for leader in leaders_per_country[country]:
        wikipedia_url = leader["wikipedia_url"]
        first_paragraph = get_first_paragraph(wikipedia_url)
        print(first_paragraph)

    

François Gérard Georges Nicolas Hollande is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.
Nicolas Paul Stéphane Sarközy de Nagy-Bocsa is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilty of having tried to bribe a judge in 2014 to obtain information and spending beyond legal campaign funding limits during his 2012 re-election campaign.
François Maurice Adrien Marie Mitterrand was a French politician and statesman who served as President of France from 1981 to 1995, the longest holder of that position in the history of France. As a former Socialist Party F

## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [110]:
# < 20 lines
def get_leaders():
    # define the urls
    root_url ="https://country-leaders.onrender.com/"
    cookie_url = "cookie"
    countries_url = "countries"
    leaders_url = "leaders"
    # get the cookies
    cookie_response = requests.get(f"{root_url}/{cookie_url}")
    cookies = cookie_response.cookies.get_dict()
    # get the countries
    countries = requests.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    # loop over them and save their leaders in a dictionary
    leaders_dict = {}
    for country in countries:
        leaders = requests.get(f"{root_url}/{leaders_url}", cookies=cookies,params={"country":country}).json()
        for leader in leaders:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph(wikipedia_url)
            key = leader["first_name"] + " " + leader["last_name"]
            leaders_dict[key] = first_paragraph
            
    return leaders_dict





In [111]:
# Check the output of your function (2 lines)
print(get_leaders())

{'François Hollande': 'François Gérard Georges Nicolas Hollande is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.', 'Nicolas Sarkozy': 'Nicolas Paul Stéphane Sarközy de Nagy-Bocsa is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilty of having tried to bribe a judge in 2014 to obtain information and spending beyond legal campaign funding limits during his 2012 re-election campaign.', 'François Mitterrand': 'François Maurice Adrien Marie Mitterrand was a French politician and statesman who served as President of France from 1981 to 1995, the longest holder o

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines

def get_fresh_cookies(root_url):
    response = requests.get(f"{root_url}/cookie")
    return response.cookies.get_dict()

def get_leaders():
    # define the urls
    root_url ="https://country-leaders.onrender.com/"
    countries_url = "countries"
    leaders_url = "leaders"
    # get the countries
    countries = requests.get(f"{root_url}/{countries_url}", cookies=get_fresh_cookies(root_url)).json()
    # loop over them and save their leaders in a dictionary
    leaders = {}
    for country in countries:
        leaders_response = requests.get(f"{root_url}/{leaders_url}", cookies=get_fresh_cookies(root_url),params={"country":country})
        if leaders_response.status_code == 403:
            leaders_response = requests.get(f"{root_url}/{leaders_url}", cookies=get_fresh_cookies(root_url),params={"country":country})
        leaders_list = leaders_response.json()
        for leader in leaders_list:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph(wikipedia_url)
            key = leader["first_name"] + " " + leader["last_name"]
            leaders[key] = first_paragraph
    return leaders


Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
print(get_leaders())

{'George Washington': 'George Washington was a Founding Father and the first president of the United States, serving from 1789 to 1797. As commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War against the British Empire. He is commonly known as the Father of the Nation for his role in bringing about American independence.', 'Barack Obama': 'Barack Hussein Obama II is an American politician who was the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president in American history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.', 'Abraham Lincoln': 'Abraham Lincoln was the 16th president of the United States, serving from 1861 until his assassination in 1865. He led the United States through the American Civil War, defeating the Confederate States of America and playing a ma

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines
from requests import Session

# def session_request(url: str, session: Session):
#     # Instead of using request.get, we use our session
#     response = session.get(url)
#     response_json = response.json()
#     for character in  response_json["data"]:
#         print(character["attributes"]["name"])


def get_fresh_cookies(root_url):
    session = requests.Session()
    response = session.get(f"{root_url}/cookie")
    return response.cookies.get_dict()

def get_leaders():
    # define the urls
    root_url ="https://country-leaders.onrender.com"
    countries_url = "countries"
    leaders_url = "leaders"
    # get the countries
    session = requests.Session()
    cookies = get_fresh_cookies(root_url, session)
    countries = session.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    # loop over them and save their leaders in a dictionary
    leaders = {}
    for country in countries:
        leaders_response = session.get(f"{root_url}/{leaders_url}", cookies=cookies,params={"country":country})
        if leaders_response.status_code == 403:
            leaders_response = session.get(f"{root_url}/{leaders_url}", cookies=get_fresh_cookies(root_url, session),params={"country":country})
        leaders_list = leaders_response.json()
        for leader in leaders_list:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph(wikipedia_url)
            key = leader["first_name"] + " " + leader["last_name"]
            leaders[key] = first_paragraph
    return leaders


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [122]:
# <25 lines
def get_fresh_cookies(root_url):
    session = requests.Session()
    response = session.get(f"{root_url}/cookie")
    return response.cookies.get_dict()

def get_leaders():
    # define the urls
    root_url ="https://country-leaders.onrender.com"
    countries_url = "countries"
    leaders_url = "leaders"
    # get the countries
    session = requests.Session()
    cookies = get_fresh_cookies(root_url)
    countries = session.get(f"{root_url}/{countries_url}", cookies=cookies).json()
    # loop over them and save their leaders in a dictionary
    leaders = {}
    
    for country in countries:
        leaders_response = session.get(f"{root_url}/{leaders_url}", cookies=cookies,params={"country":country})
        if leaders_response.status_code == 403:
            leaders_response = session.get(f"{root_url}/{leaders_url}", cookies=get_fresh_cookies(root_url),params={"country":country})
        leaders_list = leaders_response.json()
        leaders[country] = []
        for leader in leaders_list:
            wikipedia_url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph(wikipedia_url)
            name = leader["first_name"] + " " + leader["last_name"]
            leaders[country].append({
            "name": name,
            "description": first_paragraph
        })
    return leaders


print(get_leaders())



{'fr': [{'name': 'François Hollande', 'description': 'François Gérard Georges Nicolas Hollande is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.'}, {'name': 'Nicolas Sarkozy', 'description': 'Nicolas Paul Stéphane Sarközy de Nagy-Bocsa is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilty of having tried to bribe a judge in 2014 to obtain information and spending beyond legal campaign funding limits during his 2012 re-election campaign.'}, {'name': 'François Mitterrand', 'description': 'François Maurice Adrien Marie Mitterrand was a French politician and st

Test your new functions.



In [123]:
print(get_leaders())

{'fr': [{'name': 'François Hollande', 'description': 'François Gérard Georges Nicolas Hollande is a French politician who served as President of France from 2012 to 2017. Before his presidency, he was First Secretary of the Socialist Party from 1997 to 2008, Mayor of Tulle from 2001 to 2008, as well as President of the General Council of Corrèze from 2008 to 2012. He has also held the 1st constituency of Corrèze seat in the National Assembly three times, first from 1988 to 1993, then from 1997 to 2012, and from 2024 onwards.'}, {'name': 'Nicolas Sarkozy', 'description': 'Nicolas Paul Stéphane Sarközy de Nagy-Bocsa is a French politician who served as President of France from 2007 to 2012. In 2021, he was found guilty of having tried to bribe a judge in 2014 to obtain information and spending beyond legal campaign funding limits during his 2012 re-election campaign.'}, {'name': 'François Mitterrand', 'description': 'François Maurice Adrien Marie Mitterrand was a French politician and st

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json
import os
leaders_per_country = get_leaders()
filename = "leaders.json"
path = os.path.abspath("")
access_file = os.path.join(path, "data", filename)
os.makedirs(os.path.dirname(access_file), exist_ok=True)

with open(access_file, 'w') as json_file:
	json.dump(leaders_per_country, json_file)


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open(access_file, 'r') as json_file:
	data = json.load(json_file)
for key, value in data.items():
    print(f"{key} : {value}")

us : [{'name': 'George Washington', 'description': 'George Washington was a Founding Father and the first president of the United States, serving from 1789 to 1797. As commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War against the British Empire. He is commonly known as the Father of the Nation for his role in bringing about American independence.'}, {'name': 'Barack Obama', 'description': 'Barack Hussein Obama II is an American politician who was the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president in American history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.'}, {'name': 'Abraham Lincoln', 'description': 'Abraham Lincoln was the 16th president of the United States, serving from 1861 until his assassination in 1865. He led the United States through the A

Make a function `save(leaders_per_country)` to call this code easily.

In [100]:
# 3 lines
import os
import json

def save(leaders_per_country, filename):
   path = os.path.abspath("")
   access_file = os.path.join(path, "data", filename)
   os.makedirs(os.path.dirname(access_file), exist_ok=True)
   with open(access_file, 'w') as json_file:
       json.dump(leaders_per_country, json_file)

def read(filename):
    filename = "leaders.json"
    path = os.path.abspath("")
    access_file = os.path.join(path, "data", filename)
    with open(access_file, 'r') as json_file:
	    data = json.load(json_file)
    return data

def display(filename):
    data = read(filename)
    for key, value in data.items():
        print(f"{key} : {value}")
       




    

In [101]:
# Call the function (1 line)
filename = "leaders.json"
save(leaders_per_country, filename)
display(filename)


fr : [{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url

## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!